In [101]:
import pandas as pd
import numpy as np
import time
import pickle

from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, RandomForestRegressor
from sklearn.preprocessing import normalize, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline, make_pipeline

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

# fix random seed for reproducibility
SEED = 42
np.random.seed(SEED)

DATA_DIR = '/home/memer/dev/ml/kaggle7/data/'

# features numériques
FEAT_NUM_IN = ['libelle_plaquette', 'libelle_ampoule', 'libelle_flacon', 'libelle_tube', 'libelle_stylo', 'libelle_seringue',
            'libelle_pilulier', 'libelle_sachet', 'libelle_comprime', 'libelle_gelule', 'libelle_film', 'libelle_poche',
            'libelle_capsule'] + ['nb_plaquette', 'nb_ampoule', 'nb_flacon', 'nb_tube', 'nb_stylo', 'nb_seringue',
            'nb_pilulier', 'nb_sachet', 'nb_comprime', 'nb_gelule', 'nb_film', 'nb_poche', 'nb_capsule', 'nb_ml']
# features date
FEAT_DATE_IN = ['date declar annee', 'date amm annee']
# features catégorielles
FEAT_CAT_IN = ['statut', 'etat commerc', 'agrement col', 'tx rembours', 'statut admin', 'type proc']
# features texte
FEAT_TEXT_IN = ['libelle', 'substances', 'voies admin', 'titulaires', 'forme pharma']

FEAT_ALL_IN = FEAT_NUM_IN + FEAT_DATE_IN + FEAT_CAT_IN + FEAT_TEXT_IN


In [102]:

def mape_error(log_y_true, log_y_pred): 
    # type: (Series, Series) -> Series
    y_true = np.exp(log_y_true)
    y_pred = np.exp(log_y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_scorer = make_scorer(mape_error, greater_is_better = False)

def grid_search_mape(estimator, X, Y, parameters, nb_folds):
    print("Performing grid search...")
    grid_search = GridSearchCV(estimator, parameters, scoring=mape_scorer, cv=nb_folds, n_jobs=-1)
    
    print "pipeline:   "  + str([name for name, _ in pipeline.steps])
    t0 = time.time()
    grid_search.fit(X, Y)
    print "=> done in %0.3fs" % (time.time() - t0)
    print "Best score: %0.3f" % grid_search.best_score_
    print "Best parameters set:"
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    res = grid_search.cv_results_
    print "Results by parameters:"
    for i in range(0, len(res.get('params'))):
        p = res.get('params')[i]
        m = res.get('mean_test_score')[i]
        s = res.get('std_test_score')[i]
        print p
        print '\tMape: %.2f\t(std: %.2f)' % (m,s)

def displayUniqueCount(series):
    # type: (Series) -> DataFrame
    data = np.unique(series, return_counts=True)
    df = pd.DataFrame(
        data = {'Valeur':data[0], 'Nombre':data[1]},
        columns = ['Valeur', 'Nombre'])
    df.sort_values(by ="Nombre", ascending=False, inplace=True)
    return df

def agg_duplicate_rows(df, feats_gb, feat_target, agg_func):
    # type: (DataFrame, *str, str, function) -> DataFrame
    mean_target_df = df.groupby(feats_gb)[feat_target].agg({'mean_target' : agg_func})
    result_df = (df
                 .drop_duplicates(feats_gb)
                 .set_index(feats_gb)
                 .join(mean_target_df)
                 .reset_index(level=feats_gb)
                 .drop(feat_target, axis=1)
                 .rename(columns={'mean_target': feat_target})
                )
    return result_df

def addNumColsFromLibelle(df, libelles_to_extract):
    # type: (DataFrame, str) -> *str
    new_feats_col = []
    
    for lib in libelles_to_extract:
        nb_col_name = 'nb_' + lib
        lib_col_name = 'libelle_' + lib
        pattern = r'([0-9]*) ?' + lib
        
        df[nb_col_name] = (df['libelle']
                                .str.extract(pattern, expand=False) # extract pattern 
                                .str.replace(r'^$', '1')            # replace match with empty group by 1
                                .fillna(0)                          # replace mismatch by 0
                                .astype(int)                        # convert string to int
                                )
        df[lib_col_name] = df.libelle.apply(lambda x: 1 if lib in x else 0)

        new_feats_col.append(nb_col_name)
        new_feats_col.append(lib_col_name)
        
    return new_feats_col

def kfold_validation(X, Y, regressor):
        error_global = 0
        NBROUND = 5
        kf = KFold(n_splits=NBROUND, shuffle=True, random_state=SEED)
        for train_index, test_index in kf.split(X):
            start = time.time()
            X_train, X_test = X.ix[train_index, :], X.ix[test_index, :]
            Y_train, Y_test = Y[train_index], Y[test_index]
            
            regressor.fit(X_train, Y_train)
            pred = regressor.predict(X_test)
            
            error_fold = mape_error(Y_test, pred)
            error_global += error_fold / NBROUND
            print str(error_fold) + ' duration: ' + str(time.time() - start)
            
        print 'MAPE Error : ' + str(error_global)
        return error_global

# Import des données
 - ```train``` : 8564 medicaments / 41 variables
 - ```test``` : 3671 medicaments 

In [103]:
train = pd.read_csv(DATA_DIR + 'boites_medicaments_train.csv',encoding='utf-8',sep=';')
test = pd.read_csv(DATA_DIR + 'boites_medicaments_test.csv',encoding='utf-8',sep=';')

# Preparation des donnees

### Extraction de libellés

In [ ]:
libelles_to_extract = ['cartouche', 'bouteille', 'inhalateur', 'multidose']
feat_num_new = addNumColsFromLibelle(train, libelles_to_extract)
addNumColsFromLibelle(test, libelles_to_extract)


### doublons

In [105]:
feats_groupby = FEAT_ALL_IN + feat_num_new
feat_target = 'prix'

train = agg_duplicate_rows(train, feats_groupby, feat_target, np.mean)

### Encodage des features catégorielles

Les algorithmes de machine learning s'attendent à avoir en entrée des nombres, et non pas des chaînes de caractères. C'est pourquoi nous transformons les features catégorielles en nombres, à l'aide de LabelEncoder()

In [106]:
for c in FEAT_CAT_IN:
    le = LabelEncoder()
    le.fit(train[c].append(test[c]))
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])

 ### Split des catégories multi-valeurs
 Les catégories dont les valeurs sont des listes d'éléments sont développées sous forme de n catégories binaires (n étant le nombre d'éléments distincts pour la catégorie dans le dataset) 

In [107]:
train['substances_clean'] = (train['substances'].str.replace(r'\([^\)]*\)', ''))
test['substances_clean'] = (test['substances'].str.replace(r'\([^\)]*\)', ''))

In [108]:
def expandedOHE(train_df, test_df, colName):
    # type: (DataFrame, DataFrame, str) -> *str
    distinctCategs = (train_df[colName]
                      .apply(lambda st : st.split(','))
                      .apply(pd.Series)
                      .unstack()
                      .dropna()
                      .str.strip()
                      .unique())
    for categorie in distinctCategs:
        train_df[categorie] = train_df[colName].apply(lambda x : 1 if categorie in x else 0)
        test_df[categorie] = test_df[colName].apply(lambda x : 1 if categorie in x else 0)
    return list(distinctCategs) 

## le split de la categorie "substances" permet de passer de 45 à 32%
feat_substances = expandedOHE(train, test, 'substances_clean')
                            
## le split de la categorie "voies admin" dégrade l'estimation
feat_substances = expandedOHE(train, test, 'substances') 




### Quantites et prix unitaire (prix/quantite)

In [109]:
train['logprix'] = train['prix'].apply(np.log)

# Creation d'un modele

### Regressor : Grid search with cross val

In [111]:
feats = FEAT_NUM_IN + FEAT_CAT_IN + feat_substances + feat_num_new + feat_voies_admin

Y = train['logprix']
X = train[feats]

#GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor, RandomForestRegressor
reg = RandomForestRegressor(n_jobs=-1, criterion='mse', random_state=SEED) 
pipeline = make_pipeline(StandardScaler(), reg) #, 

parameters = {'randomforestregressor__n_estimators':  [10, 50, 100], 
              'randomforestregressor__min_impurity_split': [1e-5, 1e-6]}

grid_search_mape(pipeline, X, Y, parameters, 5)

Performing grid search...
pipeline:   ['standardscaler', 'randomforestregressor']
=> done in 24.320s
Best score: -32.661
Best parameters set:
	randomforestregressor__min_impurity_split: 1e-05
	randomforestregressor__n_estimators: 50
Results by parameters:
{'randomforestregressor__n_estimators': 50, 'randomforestregressor__min_impurity_split': 1e-05}
	Mape: -32.66	(std: 1.43)


### Regressor : Simple crossval

In [31]:
reg = RandomForestRegressor(n_jobs=-1, random_state=SEED, n_estimators=50, min_impurity_split=1e-05) 
kfold_validation(X,Y,reg)

31.2704234689 duration: 1.05395293236
33.5823166283 duration: 0.926917076111
33.813629656 duration: 1.06042194366
31.1133842608 duration: 0.963861942291
32.9382171978 duration: 1.01814508438
MAPE Error : 32.5435942423


32.543594242348206

### Neural network

In [112]:
# split into input (X) and output (Y) variables
feats = FEAT_NUM_IN + FEAT_CAT_IN + feat_substances + feat_num_new

# create model
# hyperopt / bash normalization / dropout / normalization entre les couches / 800 epoch / batch 50 / early stopping 
def create_model():
    # create model
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(len(feats),)))
    model.add(Dense(100, input_dim=len(feats), init='normal', activation='relu', W_constraint=maxnorm(5)))
    model.add(BatchNormalization())
    #model.add(Dense(15, init='normal', activation='relu'))
    model.add(Dense(50, init='normal', activation='sigmoid', W_constraint=maxnorm(5)))
    model.add(BatchNormalization())
    model.add(Dense(1, init='normal', W_constraint=maxnorm(5)))
    # Compile model
    model.compile(loss='mape', optimizer='rmsprop')
    return model


### Grid search avec Kerasrgressor

In [ ]:
X = train[feats]
Y = train['logprix']
reg = KerasRegressor(build_fn=create_model, nb_epoch=10, batch_size=50, verbose=0)
pipeline = make_pipeline(StandardScaler(), reg)

parameters = {'kerasregressor__nb_epoch':  [10, 100, 1000], 'kerasregressor__batch_size': [50, 100]}
grid_search = grid_search_mape(pipeline, X, Y, parameters, 5)

'''
kfold = KFold(n_splits=5, shuffle=True, random_state=SEED)
results = cross_val_score(pipeline, X, Y, cv=kfold, scoring=mape_scorer, verbose=1)
print ("\nResults: %.2f (%.2f) MAPE" % (results.mean(), results.std()))
'''

### Crossval sans Kerasregressor
Pour comparer les perfs

In [55]:
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.constraints import maxnorm

def nn_kfold_validation(X, Y, model_creator):
        error_global = 0
        NBROUND = 5
        kf = KFold(n_splits=NBROUND, shuffle=True, random_state=SEED)
        for train_index, test_index in kf.split(X):
            start = time.time()
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Y[train_index], Y[test_index]
            
            model = None
            model = model_creator()
            model.fit(X_train, Y_train, nb_epoch=500, batch_size=50, verbose=0)
            pred = model.predict(X_test)
            
            error_fold = mape_error(Y_test, pred[0])
            error_global += error_fold / NBROUND
            print str(error_fold) + ' duration: ' + str(time.time() - start)
            
        print 'MAPE Error : ' + str(error_global)
        return error_global
    
X_scaler = StandardScaler()
X = X_scaler.fit_transform(train[feats])
Y = train['logprix']

nn_kfold_validation(X, Y, create_model)

# Predictions et soumission

In [63]:
# configure regressor with best params from grid_search
reg = RandomForestRegressor(n_jobs=-1, random_state=SEED, criterion='mae', n_estimators=50, min_impurity_split=1e-05) 
pipeline = make_pipeline(StandardScaler(), reg)
# fit on full train dataset
pipeline.fit(train[feats], train['logprix'])

# predicttest prices
predictions = np.exp(pipeline.predict(test[feats]))

# write to soumission.csv
pd.DataFrame(predictions, index=test['id']).to_csv(DATA_DIR + 'soumission.csv',  
                          header=['prix'],
                          sep = ';')

### Enregistrement

In [15]:
train.to_csv(DATA_DIR + 'train_df.csv', encoding='utf-8', sep=';')
test.to_csv(DATA_DIR + 'test_df.csv', encoding='utf-8', sep=';')

with open(DATA_DIR + 'substances.pkl', 'wb') as f:
    pickle.dump(feat_substances, f)

### Chargement

In [6]:
# chargement des datasets préparés
train = pd.read_csv(DATA_DIR + 'train_df.csv',encoding='utf-8',sep=';')
test = pd.read_csv(DATA_DIR + 'test_df.csv',encoding='utf-8',sep=';')

with open(DATA_DIR + 'substances.pkl', 'rb') as f:
    feat_substances = pickle.load(f)